# LTsim Use-Case Example: Benchmarking machine learning models

The methods in this script include: standardizing data formats across our datasets, sets up the ML models, and calculates case/control AUCs as well as feature-selection AUCs

In [1]:
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2

In [3]:
import glob

import argparse
import joblib
from pathlib import Path
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_validate

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics

import matplotlib.pyplot as plt 

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
# from heritability_model_interpretable import *

In [94]:
class ModelSum:
    """Handy class to store all information about each model; primarily useful if running each model in parellel and need to save output"""
    def __init__(self, pset, simnum):
        self.pset = pset
        self.simnum = int(simnum)

        self.data_type = None
        self.rho=None
        self.pve=None
        self.maf_frac=None
        self.obs=None
        self.k=None
        self.overlap=None
        self.class_imbalance = 0.5

        self.real_interp_auc = {}
        self.rand_interp_auc = {}

        self.real_class_auc = {}
        self.rand_class_auc = {}

        self.classification_aucs = {}
        ### Return importance scores as well
        self.vscores = {}

    def populate_pset_vars(self):
        # Set up simlation run
        sp = self.pset.split('-')

        if 'prop_case' in sp:
            self.class_imbalance = float(sp[sp.index('prop_case')+1])
        if 'rho' in sp:
            self.rho = float(sp[sp.index('rho')+1])
        if 'pve' in sp:
            self.pve = float(sp[sp.index('pve')+1])
        if 'maf_frac' in sp:
            self.maf_frac = float(sp[sp.index('maf_frac')+1])
        if 'obs' in sp:
            self.obs = float(sp[sp.index('obs')+1])
        if 'k' in sp:
            self.k = float(sp[sp.index('k')+1])
        if 'ind' in sp:
            self.overlap = True
        elif 'non_overlap_degree' in sp:
            self.overlap = False


def split_data(X, y, test_size=0.1, rand_state=5):
    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=test_size, random_state=rand_state,
                                                    shuffle = True, stratify = y)
    return train_x, test_x, train_y, test_y


def train_model(mod_type, train_x, test_x, train_y, test_y, var_labels=None, causal_vars=None, var_type='snps', rand_state=5, calc_interpretability=False):
    # Setup classifier - define model, the auc, and coefficients
    implemented_list = ['log_reg', 'log_reg_cv', 'rand_forest', 'rand_forest_cv']
    interp_auc = None
    vscores = None

    assert mod_type in implemented_list, 'check mod type: {}'.format(mod_type)
    if mod_type == 'log_reg':
        mlmod = LogisticRegression(penalty='l1', solver='liblinear', class_weight='balanced', 
                                   random_state=rand_state)
        mlmod.fit(train_x, np.ravel(train_y)) 
        coefs = np.abs(mlmod.coef_) 
        
    elif mod_type == 'log_reg_cv':
        mlmod = LogisticRegressionCV(Cs=np.logspace(-4, -1.85, 80), penalty='l1', solver='liblinear', class_weight='balanced', 
                                   random_state=rand_state, cv=3, scoring = 'roc_auc',
                                    ) #n_jobs=-1, #verbose=2
        mlmod.fit(train_x, np.ravel(train_y)) 
        coefs = np.abs(mlmod.coef_) 

    elif mod_type =='rand_forest':
        mlmod = RandomForestClassifier(class_weight='balanced', random_state=rand_state)
        mlmod.fit(train_x, np.ravel(train_y)) 
        coefs = np.abs(mlmod.feature_importances_)
        
    elif mod_type == 'rand_forest_cv':
        rfc = RandomForestClassifier(class_weight='balanced', random_state=rand_state)

        # Declare a hyperparameter grid
        # could try varying other RF hyperparameters as well
        param_grid = {
            "n_estimators": [50, 100, 500],
            "max_depth": [10, 50, None],
            "min_samples_leaf": [1 ,25],
        }

        # Perform grid search, fit it, and print score
        mlmod = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=3, scoring='roc_auc',
                             ) #n_jobs=-1, verbose=2
        mlmod.fit(train_x, np.ravel(train_y)) 
#         test_class_auc = mlmod.score(test_x, test_y)
        coefs = np.abs(mlmod.best_estimator_.feature_importances_)
    
    test_class_auc = roc_auc_score(test_y, mlmod.predict_proba(test_x)[:, 1])

    if calc_interpretability:
        vscores = pd.DataFrame(zip(var_labels, np.ravel(coefs)), columns=['var', 'score']).set_index('var')
        if causal_vars is not None:
            _, interp_auc = calc_ROC(vscores, causal_vars)
                  
    ### Return vscores as well
    return mlmod, test_class_auc, interp_auc, vscores

def calc_ROC(df_scores, true_causal):
    assert true_causal is not None, 'check true_causal list'

    npips = len(df_scores)
    all_vals = list(df_scores.index)
    Pos = true_causal
    Negs = set(all_vals) - set(Pos)

    df_res = pd.DataFrame(columns=["TPR", "FPR", "FDR", "PWR"], dtype=object) 
    df_scores.sort_values(by=['score'], ascending=False, inplace=True)
    
    for i in range(1,npips+1):
        v = df_scores[0:i].index
        z = set(all_vals) -  set(v)
        TP = len(set(v) & set(Pos))
        FP = len(set(v)-set(Pos))
        TN = len(set(z) & set(Negs))
        FN = len(set(z)-set(Negs))
        TPR = TP/(TP+FN)
            
        FPR = FP/(FP+TN) 
        FDR = FP/(FP+TP)
        PWR = 1-(1-TPR)
        df_res.loc[len(df_res)] = [TPR, FPR, FDR, PWR]
        
    auc = metrics.auc(np.array(df_res['FPR']), np.array(df_res['TPR']))
    return df_res, auc

def score_cv(model_type, X, y, true_causal=None, calc_interpretability=False, cv=5):
    mods = []
    class_aucs = []
    interp_aucs = []
    ### Keep track of vscores as well
    vscores_l = []
    
    for i in range(cv):
        train_x, test_x, train_y, test_y = split_data(X, y, rand_state=i) #check random state set
        mlmod, test_class_auc, interp_auc, vscores = train_model(model_type, train_x, test_x, train_y, test_y, true_causal, calc_interpretability=calc_interpretability) 

        mods.append(mlmod)
        class_aucs.append(test_class_auc)
        interp_aucs.append(interp_auc)
        vscores_l.append(vscores)
        
    return mods, class_aucs, interp_aucs, vscores_l

In [143]:
def get_file(dataset, simnum=None, most_variable=False):
    '''Each dataset is (unfortunately) in sligtly different format; use function to get into numpy arrays
    
    Standardize to:
    genome: np array of SNPs x patients
    phenotype: np array of 0 (controls) and 1 (cases)
    where the ordering of the patients in genome is identical to the ordering of the phenotype
    '''
    if dataset == 'als': #gzipped csvs
        data_path = '/scratch/users/velina/sparse_nn/ltsim/data/als'
        phenotype = load_data(data_path + '/EUR-AFR_y_labeled.csv').set_index('Unnamed: 0')
        if most_variable:
            genome = load_data(data_path + '/EUR-AFR_most_var_subset_100000_snps.csv.gz').set_index('Unnamed: 0')
        else:
            simnum = simnum - 1 # ltsim/epigen are 1 indexed while the real are 0 indexed; adjusted here.
            genome = load_data(data_path + '/EUR-AFR_rand_subset_100000_snps_{}.csv.gz'.format(simnum)).set_index('Unnamed: 0')
        assert (genome.columns == phenotype.index).all(), 'check ordering'
        genome = genome.values
        phenotype = (phenotype['label1']-1).values
        
    elif dataset == 'diabetes': # pkled pandas
        data_path = '/scratch/users/velina/sparse_nn/t1d_data/snp_subset'
        phenotype = load_data(data_path + '/y_labeled.csv').set_index('Unnamed: 0')
        if most_variable:
            genome = load_data(data_path + '/most_var_subset_100000_snps.p')
        else:
            simnum = simnum - 1 # ltsim/epigen are 1 indexed while the real are 0 indexed; adjusted here.
            genome = load_data(data_path + '/rand_subset_100000_snps_{}.p'.format(simnum))
        assert (genome.columns == phenotype.index).all(), 'check ordering'
        genome = genome.values
        phenotype = (phenotype['label1']-1).values
        
    elif dataset == 'epigen_pop2': # pkled numpys
        data_path = '/scratch/users/divyar/epigen/sim/epigen_ceu_asw'
        phenotype = load_data(data_path + '/ysim.{}.txt'.format(simnum))
        genome = load_data(data_path + '/Xsim.{}.npy'.format(simnum))
        
    elif dataset == 'epigen_pop1':# pkled numpys
        data_path = '/scratch/users/divyar/epigen/sim/epigen_ceu'
        phenotype = load_data(data_path + '/ysim.{}.txt'.format(simnum))
        genome = load_data(data_path + '/Xsim.{}.npy'.format(simnum))
        
    elif dataset == 'ltsim_pop1': #update with relevant path once final ltsim complete
        data_path = '/scratch/users/aconard/sims/no_overlap/non_overlap_degree-4-ind-1e+06-tot_snp_sim-5000-frac_causal-0.1-k-0.1-obs-1500-maf_frac-0.05-pve-0.6-rho-0.5'
        phenotype = load_data(data_path + '/ysim.{}.txt'.format(simnum))
        phenotype = phenotype-1
        genome = load_data(data_path + '/Xsim.{}.txt'.format(simnum)).T
        
    elif dataset == 'ltsim_pop2': #update with relevant path once final ltsim complete
        data_path = '/scratch/users/aconard/sims/no_overlap/non_overlap_degree-4-ind-1e+06-tot_snp_sim-5000-frac_causal-0.1-k-0.1-obs-1500-maf_frac-0.05-pve-0.6-rho-0.5'
        phenotype = load_data(data_path + '/ysim.{}.txt'.format(simnum))
        phenotype = phenotype-1
        genome = load_data(data_path + '/Xsim.{}.txt'.format(simnum)).T
    else:
        print('error loading files')
    return phenotype, genome

def load_data(filepath):
    filepath = Path(filepath)
    # load each datatype accordingly
    if (filepath.suffix == '.gz') | (filepath.suffix == '.csv'):
        file = pd.read_csv(filepath)
    elif (filepath.suffix)=='.p':
        file = joblib.load(filepath)
    elif (filepath.suffix == '.npy'):
        file = np.load(filepath)
    elif filepath.suffix == '.txt':
        file = np.loadtxt(filepath)
    return file

In [ ]:
df_model_results = pd.DataFrame()

model_list = ['log_reg', 'log_reg_cv', 'rand_forest', 'rand_forest_cv']
dataset_list = ['als', 'diabetes', 'epigen_pop1','epigen_pop2', 'ltsim_pop1']
simnum_max = 2 # set to number of simulations want to be analyzed.

for data in dataset_list:
    srange = list(range(1,simnum_max))
    if (data == 'als') | (data == 'diabetes'):
        srange = srange + ['most_variable'] #if ALS or diabetes, include a most variable simulation.
    for snum in srange:
        print(data, snum)
        if snum == 'most_variable':
            variable=True
        else:
            variable=False
        cpheno, cgenome = get_file(data, snum, most_variable=variable)
        print(data, cgenome.shape, cpheno.shape)
        train_x, test_x, train_y, test_y = split_data(cgenome.T, cpheno, rand_state=5)

        for mod in model_list:
            print(mod)
            #If want epigen/ltsim interpretability, can update this and pass in true causal SNPs
            true_causal=None
            calc_interpretability=False 
            model_type=mod
            mlmod, test_class_auc, interp_auc, vscores = train_model(model_type, train_x, test_x, train_y, test_y, 
                                                                     true_causal, calc_interpretability=calc_interpretability) 
            ncase = train_y.sum()+test_y.sum()
            nctr = (len(train_y)+len(test_y))-ncase
            dfnew = pd.DataFrame([{'data':data, 'mod':mod, 'sim':'sim_{}'.format(snum), 'test_auc':test_class_auc, 'num_cases': ncase, 'num_controls':nctr}])
            df_model_results = pd.concat((df_model_results, dfnew), axis = 0, ignore_index=True)    

als 1
als (100000, 516) (516,)
log_reg
log_reg_cv
rand_forest
rand_forest_cv
als most_variable
als (100000, 516) (516,)
log_reg
log_reg_cv
rand_forest
rand_forest_cv
diabetes 1
diabetes (100000, 4901) (4901,)
log_reg
log_reg_cv
rand_forest
rand_forest_cv
diabetes most_variable
diabetes (100000, 4901) (4901,)
log_reg
log_reg_cv
rand_forest
rand_forest_cv
epigen_pop1 1
epigen_pop1 (100000, 5000) (5000,)
log_reg
log_reg_cv
rand_forest
rand_forest_cv
epigen_pop2 1
epigen_pop2 (100000, 5000) (5000,)
log_reg
log_reg_cv
rand_forest
rand_forest_cv


In [13]:
# # One testcase 
# mod_list = ['log_reg', 'log_reg_cv', 'rand_forest', 'rand_forest_cv']
# ctrain_x, ctest_x, ctrain_y, ctest_y, csnp_labels = split_data(cX, cY, rand_state=4)

# for model in mod_list:
#     mmod, mcauc, miauc, mvscores = train_model(model, ctrain_x, ctest_x, ctrain_y, ctest_y) 
#     print(model, np.mean(mcauc), np.std(mcauc))

rand_forest_cv 0.4496124031007752 0.0
